# STEP 1: From GermanBERT to LernnaviBERT

<img src="./Figures/german_bert.png" alt="German BERT" width="300" height="280">

[Source of the image: https://huggingface.co/bert-base-german-cased]

In order to transform a pretrained GermanBERT model in a model able to predict the students’ answers to Multiple Choice Questions (MCQs), we decided to follow a pipeline made of three steps:
1. Finetuning GermanBERT on language modelling considering the Lernnavi dataset. The finetuned model will be renamed "LernnaviBERT" for clarity;
2. Finetuning LernnaviBERT to predict the correct answers to MCQs;
3. Further finetuning the model on the final downstream task, that is the prediction of the answers of a given student to MCQs, taking into account the embedding of the student.

In this notebook, we will perform the first step, that is the finetuning of a standard GermanBERT model on the Lernnavi dataset containing MCQs. To do so, we will follow an approach which is similar to the one which has been used to train the original BERT model. This notebook has been adapted from the HuggingFace tutorial ["Fine-tuning a masked language model"](https://huggingface.co/learn/nlp-course/chapter7/3?fw=tf).

## Theoretical motivation

The language models of the BERT family are usually extensively pretrained on a huge and varied corpus, and can therefore be effectively finetuned on a variety of downstram task without the need of further finetuning them on language modelling tasks. This approach, which is possible thanks to transfer learning, is likely to produce good results as long as the corpus used for pretraining the BERT model is not too different from the corpus used for finetuning on the downstream task (in our case, MCQs prediction). This is due to the fact that if the dataset used for finetuning on the downstream task is containing domain-specific words which were not observed while training, the model would treat these words as rare tokens, causing a rapid decrease in the resulting performance. As stated previously, we decided to consider GermanBERT as our BERT model ("dbmdz/bert-base-german-uncased" on HuggingFace Hub), which according to the model card (https://huggingface.co/dbmdz/bert-base-german-uncased) has been trained on a dataset of 16GB (2,350,234,427 tokens) that includes various sources: a Wikipedia dump, EU Bookshop corpus, Open Subtitles, CommonCrawl, ParaCrawl and News Crawl. Although the dataset used for training seems extensive, and our dataset based on Lernnavi MCQs do not seem too domain-specific, we decided to finetune GermanBERT on our dataset (domain adaptation) in order to verify whether this leads to a performance boost. An advantage of this approach is that the obtained model, LernnaviBERT, could be used on a variety of downstream tasks on the Lernnavi dataset, and the finetuned model by itself could be valuable to the scientific community in the future.

The scheme below depicts the first finetuning step that we will perform in this notebook:

<img src="./Figures/1_REPORT.png" alt="Scheme step 1" width="1000" height="280">

## STEP 1

The first aspect we will address is preprocessing the Lernnavi dataset containing MCQs to be suitable for our task. In particular, since we are interested to let the model adapt to the language used in the MCQs, we are not directly interested about preserving the entity of each question. For this reason, we will concatenate all the samples and split the corpus in chunks with equal size (which should be smaller than the maximum context size of the model).

First of all, we mount the Google Drive to retrieve Lernnavi's dataset later on.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We now install the various required libraries (Transformers, Datasets, and Evaluate).

In [2]:
!pip install datasets accelerate evaluate transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 119.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━

Then, we import all the packages which are necessary to run the notebook.

In [3]:
from huggingface_hub import notebook_login
from transformers import AutoModelForMaskedLM, AutoTokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments, pipeline, default_data_collator
import torch
import pickle
from bs4 import BeautifulSoup
from datasets import Dataset
import collections
import numpy as np
import math

Since we want to push the finetuned model (LernnaviBERT) to HuggingFace Hub, we have to login to the Hub using HuggingFace token. It should be noted that this cell requires to be run alone (i.e. the cells should not be run all together up to here), otherwise it would be impossible to insert the token and to login successfully.

In [4]:
notebook_login()

At this point, we define the model we want to use as our starting point, and we load both the model and the tokenizer from the checkpoint. As stated above, we decided to use the GermanBERT model provided by DBMDZ, available on HuggingFace Hub and called "dbmdz/bert-base-german-uncased".

In [5]:
model_checkpoint = "dbmdz/bert-base-german-uncased"

model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at dbmdz/bert-base-german-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Just to verify how the model works, we can introduce a sample of text with a mask token ([MASK]) and verify which tokens the model predicts as the most likely completions (we will consider five completions). Since GermanBERT expects a German text, we will use as sample text the translation in German of the sentence "This is a good [MASK]".

In [6]:
text = "Das ist eine gute [MASK]."

To determine the most probable five completions, we pass the text to the model, extract the logit, and print the five candidates.

In [7]:
inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for count, token in enumerate(top_5_tokens):
    print("---- Completion ", count + 1, " ----")
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

---- Completion  1  ----
'>>> Das ist eine gute idee.'
---- Completion  2  ----
'>>> Das ist eine gute sache.'
---- Completion  3  ----
'>>> Das ist eine gute frage.'
---- Completion  4  ----
'>>> Das ist eine gute entscheidung.'
---- Completion  5  ----
'>>> Das ist eine gute nachricht.'


The most probable five completions are the following (the translation has been added for clarity):
1. Das ist eine gute idee. -> This is a good IDEA.
2. Das ist eine gute sache. -> This is a good THING.
3. Das ist eine gute frage. -> This is a good QUESTION.
4. Das ist eine gute entscheidung. -> This is a good DECISION.
5. Das ist eine gute nachricht. -> This is a good NEWS.

At this point, we load the dataset that we will use in order to finetune the model on the language modelling task. In particular, we will consider a dataset we processed previously containing MCQs from Lernnavi. This dataset, among the others, contains the field "question" (indicating the text of each MCQ) and the field "choices" (indicating the possible choices among a user can choose from). After loading the dataset, we only keep the two columns described, and we use 'BeautifulSoup' to remove the HTML tags.

In [8]:
# Step 1: Load the dataset
with open('/content/drive/MyDrive/MLBD/datasets/Lernnavi_MCQs.pkl', 'rb') as file:
    data = pickle.load(file)

# Step 2: Select which columns to keep
data = data[["question", "choices"]].reset_index(drop = True)

# Step 3: Remove HTML tags
data["question"] = data["question"].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
data["choices"] = data["choices"].apply(lambda x: [BeautifulSoup(a, "html.parser").get_text() for a in x])

# Step 4: Convert the dataset to the required format
data = Dataset.from_pandas(data)

<ipython-input-8-7070e42e477c>:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  data["choices"] = data["choices"].apply(lambda x: [BeautifulSoup(a, "html.parser").get_text() for a in x])


We now tokenize our dataset, without truncating the individual samples (since we will split them later). It should be noted that we concatenate both the question and the choices as a preprocessing step, in order to tokenize the whole text of each MCQ. The tokenized dataset is saved in the variable 'tokenized_datasets'.

In [9]:
def tokenize_function(examples):
    to_tokenize = examples["question"]
    for answer in examples["choices"]:
        to_tokenize += answer
    result = tokenizer(to_tokenize)
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

tokenized_datasets = data.map(
    tokenize_function, batched=True, remove_columns=["question", "choices"]
)

Map:   0%|          | 0/138149 [00:00<?, ? examples/s]

At this point, we have to group all the samples and split them into chunks. In order to determine the length of each chunk, we have to take into account the maximum context size of the model (512) and the amount of GPU memory available. It should also be taken into account that a small chunk size may not be the best choice, as the context would not be taken into account during the fine-tuning. Since we will run this notebook on Colab, and the amount of GPU memory should not be an issue, we set the chunk size to 256, which is half the maximum context size of the model.

In [10]:
chunk_size = 256

To show how the concatenation works, we can consider the first twenty tokenized samples (MCQs) and verify the number of tokens they contain.

In [11]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets[:20]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> MCQ {idx} length: {len(sample)}'")

'>>> MCQ 0 length: 28'
'>>> MCQ 1 length: 28'
'>>> MCQ 2 length: 28'
'>>> MCQ 3 length: 28'
'>>> MCQ 4 length: 28'
'>>> MCQ 5 length: 28'
'>>> MCQ 6 length: 28'
'>>> MCQ 7 length: 28'
'>>> MCQ 8 length: 28'
'>>> MCQ 9 length: 28'
'>>> MCQ 10 length: 28'
'>>> MCQ 11 length: 28'
'>>> MCQ 12 length: 28'
'>>> MCQ 13 length: 28'
'>>> MCQ 14 length: 28'
'>>> MCQ 15 length: 28'
'>>> MCQ 16 length: 28'
'>>> MCQ 17 length: 28'
'>>> MCQ 18 length: 28'
'>>> MCQ 19 length: 28'


We then concatenate the samples by using a dictionary comprehension and we verify the total length in tokens.

In [12]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated MCQs length: {total_length}'")

'>>> Concatenated MCQs length: 560'


At this point, we split the MCQs into chunks whose size is 256 (according to what has been indicated in the variable 'chunk_size'). We can then verify the length of each individual chunk.

In [13]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 256'
'>>> Chunk length: 256'
'>>> Chunk length: 48'


At this point, we define a function to create chunks of length 256 starting from the complete MCQs dataset. As we noticed above, the last chunk may be smaller than 256, and the approach chosen here is to simply discard it.

In [14]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

An important aspect about the function 'group_texts()' is that it creates a new column called "labels" which, for the moment, is a copy of the "input_ids" one. This is due to the fact that we are interested in masking some tokens to let the model predict them, and we will therefore need the ground truth while training.

At this point, we create a new dataset containing the chunks as samples.

In [15]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)

Map:   0%|          | 0/679331 [00:00<?, ? examples/s]

We are still missing an important step at this point, that is masking some of the tokens in each chunk in order to finetune the model on a language modelling task. In order to do this, we consider a data collator which is suited for this task, 'DataCollatorForLanguageModeling'. We will pass to this data collator the tokenizer and an argument indicating what fraction of the tokens to mask. Following a common choice in the literature, we will mask 15% of the token (which is also the amount used to train BERT).

In [16]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

We can verify how the random masking works by feeding some chunks to the data collator. 

In [17]:
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] markiere [MASK] satze mit der [MASK]n kommasetzung als " richtig [MASK] und die ubrigen als " falsch ". [SEP] [CLS] markiere die satze mit der korrekten kommasetzung als " richtig " und die ubrigen als " [MASK] ". [SEP] [CLS] markiere diedefope mit der korrekten kommasetzung als " richtig " und die ubrigen als " falsch ". [SEP] [CLS] markiere die satze mit der korrekten kommasetz [MASK] als " [MASK] ein und die [MASK]rigen als " falsch ". [SEP] [CLS] markiere die satze mit der korrekte [MASK] kommasetzung als " richtig " und die ubrigen als " falsch ". [SEP] [CLS]wändeniere die satze mit [MASK] [MASK]n kommasetzung als " richtig " [MASK] [MASK] ubrigen als " falsch ". [SEP] [CLS] [MASK]iere die [MASK]e [MASK] der korrekten kommasetzung als " richtig " und die ubrigen miet " falsch ". [SEP] [CLS] markiere die satze mit der korrekten kommasetzung als " richtig " und die [MASK]rigen als [MASK] falsch ". [SEP] [CLS] markiere die satze mit der korrekten [MASK]asetzung [MASK] " r

From the examples, we can verify that the '[MASK]' token has been randomly inserted at various locations in the MCQs. These will be the tokens that the model will have to predict while training.

While masking individual tokens is effective to finetune a model on a language modelling task, an alternative approach could be to mask whole words instead of tokens (whole word masking). While we will consider the first approach (i.e. masking tokens), we introduce here a custom data collator that would allow to finetune the model by masking whole words instead of tokens.

In [18]:
wwm_probability = 0.2

def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

From now on, the procedure considered is standard. We start by defining the train size and the test size, and we split the dataset in a training dataset and a test dataset.

In [19]:
train_size = int(len(lm_datasets) * 0.8)
print(f"'>>> Train size: {train_size}'")
test_size = len(lm_datasets) - train_size
print(f"'>>> Test size: {test_size}'")

train_test_dataset = lm_datasets.train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)

'>>> Train size: 38476'
'>>> Test size: 9620'


We then specify the training arguments. It should be noticed that we indicate at this point that we would like to push the finetuned model to HuggingFace Hub, and the name of the model to be saved.

In [20]:
batch_size = 16
# Show the training loss with every epoch
logging_steps = len(train_test_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"LernnaviBERT",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)

The next step is to define the trainer.

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_dataset["train"],
    eval_dataset=train_test_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/lucazed/LernnaviBERT into local empty directory.


Since we are interested in quantitatively evaluating the model before and after the finetuning, we compute the perplexity of the pretrained model. In this context, a low perplexity indicates that the model is not "surprised" or "perplexed" by the unseen examples (and, as a consequence, that it has learned the basic patterns of grammar on the corpus considered). In this case, we define the perplexity as the exponential of the cross-entropy loss.

In [22]:
eval_results = trainer.evaluate()
print(f">>> Perplexity (before fine-tuning): {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity (before fine-tuning): 1.20


We are finally ready to start the training, in order to finetune our model.

In [23]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.038000,0.012825
2,0.013800,0.008268
3,0.009600,0.007105


TrainOutput(global_step=7215, training_loss=0.02043552272775606, metrics={'train_runtime': 3210.1899, 'train_samples_per_second': 35.957, 'train_steps_per_second': 2.248, 'total_flos': 1.519070539843584e+16, 'train_loss': 0.02043552272775606, 'epoch': 3.0})

After training, we can recompute the perplexity to verify whether it is smaller than the one obtained previously.

In [24]:
eval_results = trainer.evaluate()
print(f">>> Perplexity (after fine-tuning): {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity (after fine-tuning): 1.01


The perplexity is indeed smaller, indicating that the model has learnt the basic patterns of the corpus we considered. This is a useful indication that the finetuning process may have been successful.

At this point, we can push the model to the HuggingFace Hub, so that we will be able to retrieve it while performing the next steps of our pipeline.

In [25]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/420M [00:00<?, ?B/s]

Upload file runs/Jun02_20-10-47_1e075776511e/events.out.tfevents.1685739983.1e075776511e.589.2:   0%|         …

Upload file runs/Jun02_20-10-47_1e075776511e/events.out.tfevents.1685736713.1e075776511e.589.0:   0%|         …

To https://huggingface.co/lucazed/LernnaviBERT
   2ef363b..aab1932  main -> main

   2ef363b..aab1932  main -> main

To https://huggingface.co/lucazed/LernnaviBERT
   aab1932..08a821d  main -> main

   aab1932..08a821d  main -> main



'https://huggingface.co/lucazed/LernnaviBERT/commit/aab19325da6162aca526aa0e864f10800c537925'

At this point, we demonstrate how the model which has been just saved to HuggingFace hub can be retrieved, and can be used to predict the most probable completion of the text we considered at the beginning of the notebook.

In [26]:
mask_filler = pipeline(
    "fill-mask", model="lucazed/LernnaviBERT"
)

We determine the most probable completions.

In [27]:
preds = mask_filler(text)

for count, pred in enumerate(preds):
    print("---- Completion ", count + 1, " ----")
    print(f">>> {pred['sequence']}")

---- Completion  1  ----
>>> das ist eine gute sache.
---- Completion  2  ----
>>> das ist eine gute nachricht.
---- Completion  3  ----
>>> das ist eine gute wahl.
---- Completion  4  ----
>>> das ist eine gute basis.
---- Completion  5  ----
>>> das ist eine gute arbeit.


The most probable five completions are the following (the translation has been added for clarity, the suggestion obtained before finetuning is between parentheses):

1. das ist eine gute sache. -> this is a good THING. (This is a good IDEA.)
2. das ist eine gute nachricht. -> this is a good NEWS. (This is a good THING.)
3. das ist eine gute wahl. -> this is a good CHOICE. (This is a good QUESTION.)
4. das ist eine gute basis. -> this is a good BASIS. (This is a good DECISION.)
5. das ist eine gute arbeit. -> this is a good WORK. (This is a good NEWS.)

From the proposed completions, it seems clear that the model did not completely shift the predictions with respect to before (which is a positive aspect, as we don't want the model to forget the training performed up to this point). Nevertheless, the predictions are more tied to education and learning (in particular the introduction of the word 'CHOICE').

Since the first step seems to give good results, both qualitatively (visual inspection of the completions) and quantitatively (reduction of perplexity), we now proceed to the second step, that is finetuning LernnaviBERT on the prediction of the correct answers to MCQs.

**NOTE**: The complete analysis of these results is presented in the report describing the final milestone of the project.